In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
import os
import pandas as pd

In [0]:
PATH = "/content/drive/My Drive/IE481 데이터 가시화/ABC+MSBAND의 사본/"
ZIP_PATH = "/content/drive/My Drive/IE481 데이터 가시화/"

In [0]:
act_dict = {
  0:"Sleep", 
  1: "SNS",
  2 : "Video",
  3 : "Exercise",
  4 : "Not Classified",
}

In [0]:
userIds = []
for filename in os.listdir(PATH):
  if filename[0]=='P':
    userIds.append(filename.split('.')[0])

In [0]:
%cd "/content/drive/My Drive/IE481 데이터 가시화/"

/content/drive/My Drive/IE481 데이터 가시화


In [0]:
# for userId in userIds:
#   if userId in os.listdir():
#     continue
#   print(userId)
#   !mkdir $userId
#   path_to_zipfile = PATH + userId + ".zip"
#   with zipfile.ZipFile(path_to_zipfile, 'r') as zip_ref:
#     zip_ref.extractall(ZIP_PATH + userId)

P3005
P3009
P3010
P3011
P3012
P3013
P3014
P3015
P3016
P3017
P3018
P3019
P3021
P3022
P3023
P3024
P3025
P3027
P3028
P3029
P3030
P3041


In [0]:
def aggregate_dfs(keyword, l, uid):
  first = True
  df = None
  for ll in l:
    if keyword in ll:
      # print(ll)
      if first:
        df = pd.read_csv(ZIP_PATH+uid+"/"+ll)
        first = False
      else:
        df_ad = pd.read_csv(ZIP_PATH+uid+"/"+ll)
        df = pd.concat([df, df_ad])
  return df
def preprocess_mean(d):
  newd = d.copy()
  newd["new_timestamp"] = pd.to_datetime(newd["timestamp"], unit="ms")
  newd.set_index('new_timestamp', drop=True, inplace=True)
  dd = newd.resample('1H').mean()
  return dd
def preprocess_sum(d):
  newd = d.copy()
  newd["new_timestamp"] = pd.to_datetime(newd["timestamp"], unit="ms")
  newd.set_index('new_timestamp', drop=True, inplace=True)
  dd = newd.resample('1H').sum()
  return dd

In [0]:
FIRST_USER = True
for userId in userIds: 
  l = os.listdir(userId)
  print(userId)

  """## Location"""

  df = aggregate_dfs("Location", l, userId)
  if df is None: continue

  df_dropped = df.drop(columns=["altitude", "speed", "accuracy"])
  df_dropped["new_timestamp"] = pd.to_datetime(df_dropped["timestamp"], unit="ms")
  df_dropped.set_index('new_timestamp', drop=True, inplace=True)
  df_resampled = df_dropped.resample('1H').mean()

  """## Screen off"""

  df_device = aggregate_dfs("DeviceEventEntity", l, userId)
  if df_device is None: continue
  df_device["screen_type_numeric"] = df_device["type"] != "SCREEN_OFF"
  df_device_resampled = preprocess_sum(df_device)

  """## Physical Activity"""

  df_physical = aggregate_dfs("PhysicalActivityEventEntity", l, userId)
  if df_physical is None: continue
  df_physical["physical_type_numeric"] = df_physical["type"] != "STILL"
  df_physical_resampled = preprocess_sum(df_physical)

  """## Merge"""

  df_resampled = pd.merge(df_resampled, df_device_resampled, left_index=True, right_index=True, how="outer")

  """## SNS"""

  # df_sns1 = aggregate_dfs("AppUsageStatEntity", l, userId)
  # df_sns1["SNS"] = df_sns1["name"]==("Facebook" or "카카오톡" or "Instagram")
  # df_sns1_resampled = preprocess_sum(df_sns1)

  df_sns2 = aggregate_dfs("AppUsageEventEntity", l, userId)
  if df_sns2 is None: continue
  df_sns2["SNS"] = df_sns2["name"].str.contains('Facebook|카카오톡|Instagram') 
  df_sns2["Video"] = df_sns2["name"].str.contains('Youtube|Netflix|AfreecaTV|Twitch')
  df_sns2["SNS"] = df_sns2["SNS"].fillna(0)
  df_sns2["SNS"] = df_sns2["SNS"].astype(int)
  df_sns2["Video"] = df_sns2["Video"].fillna(0)
  df_sns2["Video"] = df_sns2["Video"].astype(int)
  df_sns2["FOREGROUND"] = df_sns2["type"]=="MOVE_TO_FOREGROUND"
  df_sns2["SNS_FOREGROUND"] = df_sns2["SNS"] * df_sns2["FOREGROUND"]
  df_sns2["Video_FOREGROUND"] = df_sns2["Video"] * df_sns2["FOREGROUND"]
  df_sns2_resampled = preprocess_sum(df_sns2)

  """## Exercise"""

  df_physical["Exercise"] = df_physical["type"].str.contains("WALKING|RUNNING|ON_BICYCLE")
  df_physical["Exercise"] = df_physical["Exercise"].fillna(0)
  df_physical_resampled2 = preprocess_sum(df_physical)

  """## Outside Activity"""

  all_dfs = [df_resampled, df_device_resampled, df_physical_resampled, df_physical_resampled2, df_sns2_resampled]
  df_all = all_dfs[0]
  for i, df in enumerate(all_dfs):
    if i==0: continue
    cols_to_use = df.columns.difference(df_all.columns)
    df_all = pd.merge(df_all, df[cols_to_use], left_index=True, right_index=True, how='outer')
  df_all_filtered = df_all.filter(items=['longitude', 'latitude', "screen_type_numeric", "physical_type_numeric", "Exercise", "SNS_FOREGROUND", "Video_FOREGROUND"])
  df_all_filtered["Sleep"] = df_all_filtered["screen_type_numeric"] * df_all_filtered["physical_type_numeric"]
  df_all_filtered["Sleep"].fillna(0)
  df_all_filtered["Sleep"] = df_all_filtered["Sleep"]==0
  # df_all_filtered["activity"] = df_all_filtered["activity"].astype(int)
  if df_all_filtered.dropna().empty:
    continue
  # df_SVE = df_all_filtered.filter(items=["Exercise", "SNS_FOREGROUND", "Video_FOREGROUND"])
  # df_all_filtered["maxi"] = df_SVE.max(axis=1)
  # print(df_all_filtered.columns)
  df_all_filtered["maxi"] = df_all_filtered.apply(lambda row:max(row.SNS_FOREGROUND, row.Video_FOREGROUND), axis=1)
  # df_sns2["maxi"] = df_sns2["maxi"].fillna(0)
  
  df_all_filtered["SNS"] = df_all_filtered.apply(lambda row:row.Sleep==0 and row.maxi!=0 and row.maxi==row.SNS_FOREGROUND, axis=1)
  df_all_filtered["Video"] = df_all_filtered.apply(lambda row:row.Sleep==0 and row.maxi!=0 and row.maxi==row.Video_FOREGROUND, axis=1)
  df_all_filtered["Exercise"] = df_all_filtered.apply(lambda row:row.Sleep==0 and row.maxi!=0 and row.Exercise > row.maxi, axis=1)

  df_all_filtered["Not Classified"] = df_all_filtered.apply(lambda row:row.maxi==0 and row.Exercise==0 and row.Sleep==0, axis=1)


  # for index, row in df_all_filtered.iterrows(): # we add label column and put motion type information in the column.
  #   sns = row['SNS_FOREGROUND']
  #   video = row['Video_FOREGROUND']
  #   exercise = row['Exercise']
  #   maxi = max(sns, video, exercise)
  #   if maxi==0 and row['activity']!=0:
  #     df_all_filtered.at[index, 'activity'] = 4
  #   elif maxi==sns:
  #     df_all_filtered.loc[index, 'activity'] = row['activity'] * 1
  #   elif maxi==video:
  #     print("video")
  #     df_all_filtered.loc[index, 'activity'] = row['activity'] * 2
    
  #   else:
  #     df_all_filtered.loc[index, 'activity'] = row['activity'] * 3

  df_all_filtered["userId"] = userId
  df_all_filtered['time'] = df_all_filtered.index
  df_all_filtered.reset_index(drop=True, inplace=True)

  if FIRST_USER:
    FIRST_USER = False
    final_df = df_all_filtered
  else:
    final_df = pd.concat([final_df, df_all_filtered], axis=0)

P0701
P0702
P0703
P0704
P0705
P0706
P0707
P0708
P0709
P0710
P0711
P0712
P0713
P0714
P0715
P0716
P0717
P0718
P0719
P0721
P0722
P0723
P0724
P0725
P0726
P0727
P0728
P0729
P1501
P1502
P1503
P1504
P1505
P1506
P1507
P1508
P1509
P1510
P1511
P1514
P1515
P1516
P1517
P1518
P1519
P1520
P1521
P1522
P1523
P1525
P1526
P1527
P1541
P3001
P3002
P3003
P3005
P3007
P3008
P3009


/usr/local/lib/python3.6/dist-packages/pandas/core/computation/expressions.py:178: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


P3010
P3011
P3012
P3013
P3014
P3015
P3016
P3017
P3018
P3019
P3021
P3022
P3023
P3024
P3025
P3027
P3028
P3029
P3030
P3041


In [0]:
final_df

,longitude,latitude,screen_type_numeric,physical_type_numeric,Exercise,SNS_FOREGROUND,Video_FOREGROUND,Sleep,maxi,SNS,Video,Not Classified,userId,time
0,127.347642,36.359334,6.0,0.0,False,27.0,0.0,True,27.0,False,False,False,P0701,2019-05-08 00:00:00
1,127.358407,36.364284,48.0,789.0,True,42.0,0.0,False,42.0,True,False,False,P0701,2019-05-08 01:00:00
2,127.359732,36.372619,60.0,897.0,True,63.0,0.0,False,63.0,True,False,False,P0701,2019-05-08 02:00:00
3,NaN,NaN,12.0,33.0,False,27.0,0.0,False,27.0,True,False,False,P0701,2019-05-08 03:00:00
4,NaN,NaN,0.0,0.0,False,0.0,0.0,True,0.0,False,False,False,P0701,2019-05-08 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,NaN,NaN,0.0,0.0,False,0.0,0.0,True,0.0,False,False,False,P3041,2019-05-06 19:00:00
164,127.357410,36.367545,2.0,0.0,False,0.0,0.0,True,0.0,False,False,False,P3041,2019-05-06 20:00:00
165,127.357472,36.367550,0.0,0.0,False,6.0,0.0,True,6.0,False,False,False,P3041,2019-05-06 21:00:00
166,127.357502,36.367562,NaN,0.0,False,0.0,0.0,False,0.0,False,False,True,P3041,2019-05-06 22:00:00


In [0]:
final_df.columns

Index(['longitude', 'latitude', 'screen_type_numeric', 'physical_type_numeric',
       'Exercise', 'SNS_FOREGROUND', 'Video_FOREGROUND', 'Sleep', 'maxi',
       'SNS', 'Video', 'Not Classified', 'userId', 'time'],
      dtype='object')

In [0]:
final_final_df = final_df.drop(columns=['screen_type_numeric', 'physical_type_numeric',
       'SNS_FOREGROUND', 'Video_FOREGROUND', 'maxi'])
final_final_df.head()

,longitude,latitude,Exercise,Sleep,SNS,Video,Not Classified,userId,time
0,127.347642,36.359334,False,True,False,False,False,P0701,2019-05-08 00:00:00
1,127.358407,36.364284,True,False,True,False,False,P0701,2019-05-08 01:00:00
2,127.359732,36.372619,True,False,True,False,False,P0701,2019-05-08 02:00:00
3,NaN,NaN,False,False,True,False,False,P0701,2019-05-08 03:00:00
4,NaN,NaN,False,True,False,False,False,P0701,2019-05-08 04:00:00


In [0]:
final_final_df['Sleep'].sum()

4267

In [0]:
final_final_df['Video'].sum()

26

In [0]:
final_final_df['SNS'].sum()

6788

In [0]:
final_final_df['Exercise'].sum()

5370

In [0]:
final_final_df.to_csv("dataset.csv")